In [3]:
#Download the content
import requests
#Parse 
from bs4 import BeautifulSoup
#Catch the content
from lxml import etree
#Extract content
import regex as re
#Lib spacy
import spacy
#Lib json
import json

In [4]:
RE_NORMALIZE_BLANKS = re.compile(r'[\s]+')
url = "http://www.planalto.gov.br/ccivil_03/constituicao/ConstituicaoCompilado.htm"
REQUEST_HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#Clean the text
def replace_all_problems(text):
    fim = text.replace("\r\n", " ")
    return fim

#Show objects in a better way to visualize
def pprint(dados):
    print(json.dumps(dados, indent=2, ensure_ascii=False).encode('utf-8').decode())

#Normalize the text
def normalize_text(s):
    return RE_NORMALIZE_BLANKS.sub(' ', s).strip()

#Request the html
def download_html(url):
    r = requests.get(url, headers=REQUEST_HEADER)
    return r.text

#Split the html in the end of art 250
def remove_pos_normas_centrais(html):

    return html.split("Brasília, 5 de outubro de 1988.")[0]

#Parse the soup
def parse_soup(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

#Extract the preambulo
def extract_preambulo(soup):
    dom = etree.HTML(str(soup))
    content = replace_all_problems((dom.xpath('/html/body/p[3]/font/small')[0].text))
    dictionary = {"preambulo": str(content)}
    return dictionary

def create_title(soup):
    find = soup.find_all("p", face="Arial")
    return find

html = download_html(url)
html_preambulo_e_normas_centrais = remove_pos_normas_centrais(html)
soup = parse_soup(html_preambulo_e_normas_centrais)

cf = ' '.join(p.text for p in soup.find_all('p'))
cf = normalize_text(cf)
cf = cf.strip()

In [132]:
FIND_TITLE = re.compile(r'TÍTULO\s+(?P<titulo_numero>[IXV]+)\s+(?P<titulo_descricao>.*?)(?=Art|CAPÍTULO|$)(?P<titulo_conteudo>.*?)(?=TÍTULO|$)')
FIND_CHAPTER = re.compile(r'(CAPÍTULO (?P<capitulo_numero>[IXV]+))?(?P<capitulo_descricao>.*?)(?=Art\.)(?P<capitulo_conteudo>.*?)(?=CAPÍTULO|$)')
FIND_ARTICLE = re.compile(r'Art\. (?P<artigo_numero>[123456789]+). (?P<artigo_descricao>.*?)(?=:|\.)(?P<artigo_conteudo>.*?)(?=Art\.|$)')
FIND_PARAGRAPH = re.compile(r'(Parágrafo (?P<paragrafo_numero>único))?..(?<=Parágrafo único\. )(?P<paragrafo_descricao>.*?)(?=\: I|\.)')
FIND_PARAGRAPHS = re.compile(r'((?<=\. |\).)§.(?P<paragrafo_numero>\d+))?..(?P<paragrafo_descricao>.*?)(?=§ \d+\.|CAPÍTULO|Art\.| § \d+º)')

In [147]:
principal = {'titulos': []}

for match_titulo in FIND_TITLE.finditer(cf):
    
    titulo = {
        
        'titulo_identificador': match_titulo.group('titulo_numero'),

        'titulo_descricao': match_titulo.group('titulo_descricao').strip(),

        'capitulos': []
    }
    
    principal['titulos'].append(titulo)
    
    titulo_conteudo = match_titulo.group('titulo_conteudo')
    
    for match_capitulo in FIND_CHAPTER.finditer(titulo_conteudo):
        
        capitulo = {
            'capitulo_identificador': match_capitulo.group('capitulo_numero') or '0',
            
            'capitulo_descricao': match_capitulo.group('capitulo_descricao').strip() or 'CAPÍTULO ÚNICO',
            
            'artigos': []
        }
        
        titulo['capitulos'].append(capitulo)
        
        capitulo_conteudo = match_capitulo.group('capitulo_conteudo')
        
        for match_artigo in FIND_ARTICLE.finditer(capitulo_conteudo):
            
            artigo = {
                'artigo_identificador': match_artigo.group('artigo_numero'),
                
                'artigo_descricao': match_artigo.group('artigo_descricao'),
                
                'paragrafos': []
            }
            
            capitulo['artigos'].append(artigo)
            
            artigo_conteudo = match_artigo.group('artigo_conteudo')
            
            for match_paragrafo in FIND_PARAGRAPH.finditer(artigo_conteudo):
                
                paragrafo = {

                    'paragrafo_identificador': match_paragrafo.group('paragrafo_numero'),

                    'paragrafo_descricao': match_paragrafo.group('paragrafo_descricao').strip(),

                    'incisos': []
                }

                artigo['paragrafos'].append(paragrafo)
                

pprint(principal)

{
  "titulos": [
    {
      "titulo_identificador": "I",
      "titulo_descricao": "Dos Princípios Fundamentais",
      "capitulos": [
        {
          "capitulo_identificador": "0",
          "capitulo_descricao": "CAPÍTULO ÚNICO",
          "artigos": [
            {
              "artigo_identificador": "1",
              "artigo_descricao": "A República Federativa do Brasil, formada pela união indissolúvel dos Estados e Municípios e do Distrito Federal, constitui-se em Estado Democrático de Direito e tem como fundamentos",
              "paragrafos": [
                {
                  "paragrafo_identificador": "único",
                  "paragrafo_descricao": "Todo o poder emana do povo, que o exerce por meio de representantes eleitos ou diretamente, nos termos desta Constituição",
                  "incisos": []
                }
              ]
            },
            {
              "artigo_identificador": "2",
              "artigo_descricao": "São Poderes da União, 